# Notebook to Test Three Different Methods of Downloading Files from Mega

* Method 1: Mega Download using Python Mega Library
* Method 2: Mega Download using Mega CMD Tool
* Method 3: Mega Download using Mega CMD Tool (Long Version)

## Computing Platform Check GPU (CUDA) or CPU

In [ ]:
import torch
if torch.cuda.is_available():
    device = "cuda"
else:
    print ('[WARNING] CUDA/GPU is not available! Compute-intensive scripts on this notebook will be run on CPU.')
    device =  "cpu"

## Check Environment

In [1]:
import os

if 'COLAB_GPU' in os.environ:
    print("Environment is colab")
elif 'KAGGLE_URL_BASE' in os.environ:
    print("Environment is kaggle")
else:
    print("Environment is local")

Environment is local


## Method 1: Mega Download by Using Python Mega Library

### Install Requirement

In [ ]:
%pip install mega.py

### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount._DEBUG = False
drive.mount('/content/drive', force_remount=True)

### Set Download URL and Mega ID

In [1]:
# Download URL on Mega
data_url = 'https://mega.nz/file/sBwG1DqZ#NcN7Q97CJPh5DB-tXGb3a4SV6Bubw9xPFRfqPQ4bmw8'
# Destination path for data transfer (can be in Google Drive or session storage)
destination_path = 'drive/MyDrive/MegaImport'
# Example for transfering to session storage
# transfer_to_path = './downloads/'

# Mega user name and password. Leave as empty string for anonymous login
mega_user_email = ''
mega_user_password = ''

### Download File from Mega
Download file based on download URL `data_url` to location specified in `transfer_to_path`

In [ ]:
from mega import Mega

# Create Mega User Object and Login
mega = Mega()
mega_user = mega.login(mega_user_email, mega_user_password)

# Download file specified in data_url to location specified in transfer_to_path
mega.download_url(data_url, destination_path)

## Method 2: Mega Download by Using Mega CMD Tool

### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount._DEBUG = False
drive.mount('/content/drive', force_remount=True)

### Install Requirements

In [ ]:
!apt install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https
!apt --fix-broken install

### Download and Install Mega CMD

In [ ]:
!wget https://mega.nz/linux/MEGAsync/xUbuntu_18.04/amd64/megacmd-xUbuntu_18.04_amd64.deb
!sudo dpkg -i megacmd-xUbuntu_18.04_amd64.deb

### Download File from Mega (with Progress Info)
Set download URL in Mega and destination path (in Google Drive or session storage) and download the file.

In [ ]:
import os
import contextlib
from subprocess import Popen, PIPE, STDOUT

# Download URL on Mega
data_url = 'https://mega.nz/file/sBwG1DqZ#NcN7Q97CJPh5DB-tXGb3a4SV6Bubw9xPFRfqPQ4bmw8'
# Destination path for data transfer in Google Drive
destination_path = 'drive/MyDrive/MegaImport'

# Function for printing the download progress
def print_progress(proc, stream='stdout'):
  newlines = ['\n', '\r\n', '\r']
  stream = getattr(proc, stream)
  with contextlib.closing(stream):
      while True:
          out = []
          last = stream.read(1)
          # Don't loop forever
          if last == '' and proc.poll() is not None:
              break
          while last not in newlines:
              # Don't loop forever
              if last == '' and proc.poll() is not None:
                  break
              out.append(last)
              last = stream.read(1)
          out = ''.join(out)
          yield out

# Download dataset
cmd = ["mega-get", data_url, destination_path]
proc = Popen(cmd,stdout=PIPE, stderr=STDOUT, universal_newlines=True)
for line in print_progress(proc):
    print(line)


## Method 3: Mega Download by Using Mega CMD Tool (Long Version)

### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount._DEBUG = False
drive.mount('/content/drive', force_remount=True)

### Download and Install Mega CMD

In [ ]:
import os, urllib.request
import subprocess
import contextlib
from IPython.display import clear_output
from functools import wraps
import errno
import signal
import subprocess
import glob

HOME = os.path.expanduser("~")
if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
)

if not os.path.exists("/usr/bin/mega-cmd"):
    loadingAn()
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
    clear_output()


### Download File from Mega
Specify download URL in Mega and destination path (in Google Drive or session storage) and download the file.

In [ ]:
# Set download URL on Mega
data_url = 'https://mega.nz/file/sBwG1DqZ#NcN7Q97CJPh5DB-tXGb3a4SV6Bubw9xPFRfqPQ4bmw8'
# Destination path for data transfer (in local or google drive)
destination_path = 'drive/MyDrive/MegaImport'

#It's optional to provide the MEGA username and password, it's used for giving more download quota if you have a MEGA pro account. 
MEGA_USERNAME = ""  #optional 
MEGA_PASSWORD = ""  #optional 

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def latest_file(folder):
  list_of_files = glob.glob(f'{folder}/*') # * means all 
  latest_file = max(list_of_files, key=os.path.getctime)
  return latest_file

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfer(url):
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    cmd = ["mega-get", url, destination_path]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        print(line)
        
if not destination_path:
  os.makedirs("downloads", exist_ok=True)
  destination_path = "downloads"


class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator


@timeout(10)
def runShT(args):
    return runSh(args, output=True)

def login(): 
    runShT(f"mega-login {MEGA_USERNAME} {MEGA_PASSWORD}")

#if the username and password provided then login to MEGA. 
if MEGA_USERNAME != "" and MEGA_PASSWORD != "":
    try:
        login()
    except TimeoutError:
        runSh('mega-whoami', output=True)
else:
    print("Please Input your Mega IDs.")

transfer(data_url)